In [ ]:
import logging

import climepi  # noqa
import numpy as np
from bokeh.plotting import show
from climepi import climdata, epimod

import opts
from figure_export import export_sensitivity_figure
from summary_plot import make_summary_plot

# Configure notebook
logging.getLogger().setLevel(logging.CRITICAL)

# Get options
options = opts.get_opts()
data_base_dir = options["data_base_dir"]
year_range = options["year_range"]
locations = [options["location_default"]] + options["locations_additional"]
no_locations = len(locations)
epi_model_names = [
    options["epi_model_name_default"],
    options["epi_model_name_additional"],
]
epi_model_species = [
    options["epi_model_species_default"],
    options["epi_model_species_additional"],
]
uncertainty_level = options["uncertainty_level"]
polyfit_degree = options["polyfit_degree"]

years_summary = [2040, 2060, 2080]

In [2]:
# Get climate data
ds_clim = (
    climdata.get_example_dataset("isimip_cities_daily", base_dir=data_base_dir)
    .sel(time=slice(str(year_range[0]), str(year_range[1])))
    .sel(location=locations)
)
# Yearly average temperature data
ds_temp_yearly = ds_clim.climepi.yearly_average("temperature")

In [ ]:
# Make summary plot of uncertainty in temperature
p_clim = make_summary_plot(
    ds_temp_yearly,
    data_var="temperature",
    years_summary=years_summary,
    polyfit_degree=polyfit_degree,
    uncertainty_level=uncertainty_level,
    ylim=(9, 21),
    yticks=np.arange(9, 22, 3),
    ylabel="Annual mean\ntemperature (°C)",
    legend_location="bottom_right",
)
show(p_clim)

In [ ]:
# Make summary plots of uncertainty in months suitable
panels_epi_plume = []
for epi_model_name, species_curr in zip(epi_model_names, epi_model_species):
    epi_model = epimod.get_example_model(epi_model_name)
    ds_suitability = ds_clim.climepi.run_epi_model(epi_model)
    ds_days_suitable = (
        ds_suitability["suitability"]
        .groupby("time.year")
        .sum("time")
        .rename(year="time")
        .to_dataset(name="days_suitable")
    )
    ds_days_suitable["time"] = ds_temp_yearly["time"]
    p = make_summary_plot(
        ds_days_suitable,
        data_var="days_suitable",
        years_summary=years_summary,
        polyfit_degree=polyfit_degree,
        uncertainty_level=uncertainty_level,
        ylim=(0, 360),
        yticks=np.arange(0, 361, 60),
        ylabel=f"Days suitable\n({species_curr})",
    )
    show(p)
    panels_epi_plume.append(p)

In [5]:
# Combine panels, ordered by location
panels = [p_clim] + panels_epi_plume
export_sensitivity_figure(panels)